# A Reduction Tale

> Objectives:
> * Compare operations taking place in different data containers
> * Compare sizes for these data containers
> * Help deciding when it is best to use a container or another

Let's suppose that we are going to need reductions a lot and we want to choose the best container for performing them.  First, let's start by activating our MemWatcher agent:

In [1]:
from ipython_memwatcher import MemWatcher
mw = MemWatcher()
mw.start_watching_memory()

In [1] used 0.031 MiB RAM in 0.002s, peaked 0.000 MiB above current, total RAM usage 34.965 MiB


and choose a different container for the data that we want to reduce, starting with a list:

## Regular lists

In [2]:
a = [float(i) for i in range(10*1000*1000)]

In [2] used 545.145 MiB RAM in 8.569s, peaked 70.184 MiB above current, total RAM usage 580.109 MiB


Now, proceed with a simple reduction (sum):

In [3]:
t = %timeit -o sum(a)

1 loop, best of 3: 231 ms per loop
In [3] used 0.062 MiB RAM in 0.957s, peaked 0.000 MiB above current, total RAM usage 580.172 MiB


which, in MFLOPS (Mega-FloatingPointOps-Per-Second) is:

In [4]:
print("MFLOPS:", round((len(a) / t.best / 1e6), 1))

('MFLOPS:', 43.3)
In [4] used 0.008 MiB RAM in 0.003s, peaked 0.000 MiB above current, total RAM usage 580.180 MiB


Ok, so that seems fast, but we don't have other references to compare with.  In addition, a list is not the best kind of container in terms of space consumption.  So let's try now a container that seems quite optimal in terms of memory savings.

## NumPy

In [10]:
import numpy as np

In [10] used 0.000 MiB RAM in 0.003s, peaked 0.000 MiB above current, total RAM usage 672.172 MiB


In [11]:
na = np.array(a, dtype=np.float64)

In [11] used 0.000 MiB RAM in 1.102s, peaked 0.000 MiB above current, total RAM usage 672.172 MiB


In [7]:
print("SIZE:", round((na.size * na.itemsize) / 2**20., 3))

('SIZE:', 76.294)
In [7] used 0.004 MiB RAM in 0.004s, peaked 0.000 MiB above current, total RAM usage 672.148 MiB


We see that, with 8 bytes/element, NumPy is a very efficient container.

In [8]:
t = %timeit -o sum(na)

1 loop, best of 3: 3.52 s per loop
In [8] used 0.020 MiB RAM in 14.138s, peaked 0.000 MiB above current, total RAM usage 672.168 MiB


In [9]:
print("MFLOPS:", round(len(a) / t.best / 1e6, 3))

('MFLOPS:', 2.842)
In [9] used 0.004 MiB RAM in 0.005s, peaked 0.000 MiB above current, total RAM usage 672.172 MiB


### Exercise

The performance for NumPy is several times slower than the computation with the list.  Why so?

*Hint: * We are using sum() which is a Python function.

### Solution

NumPy has a lot of overhead in producing a Python integer for every element in the array for feeding it to the sum().

*Hint:* Use internal NumPy methods (ufuncs) when possible.

In [12]:
t = %timeit -o na.sum()

10 loops, best of 3: 19 ms per loop
In [12] used 0.086 MiB RAM in 0.812s, peaked 0.000 MiB above current, total RAM usage 672.258 MiB


In [13]:
print("FLOPS:", round(len(a) / t.best / 1e6, 3))

('FLOPS:', 525.329)
In [13] used 0.004 MiB RAM in 0.004s, peaked 0.000 MiB above current, total RAM usage 672.262 MiB


This is about 100x the speed of sum() on a Python list and it is also pretty optimal in terms of both execution time and space consumed.

## Exercise

The speed in the above reduction is limited by memory speed, not CPU speed.  Could you provide a hint on the maximum memory speed that supports your laptop?

#### Solution

In [14]:
# This is an easy one.  Just divide the size of the dataset by the time that takes the reduction
(na.size * na.itemsize) / t.best / 2**30

3.914004621656657

In [14] used 0.090 MiB RAM in 0.049s, peaked 0.000 MiB above current, total RAM usage 672.352 MiB


So, in this case the memory bandwidth is close to 10 GB/s.

## Using compressed in-memory containers with bcolz

But let us suppose that we have really big data to process in our laptop and want to see if we can store our data in less space.  Enter compression:

In [15]:
import bcolz
bcolz.print_versions()
bcolz.defaults.cparams['cname'] = 'blosclz'
bcolz.defaults.cparams['clevel'] = 9
bcolz.defaults.cparams['shuffle'] = bcolz.SHUFFLE
bcolz.set_nthreads(4)

-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
bcolz version:     1.0.0rc2
NumPy version:     1.10.4
Blosc version:     1.8.0 ($Date:: 2016-03-31 #$)
Blosc compressors: ['blosclz', 'lz4', 'lz4hc', 'snappy', 'zlib']
Numexpr version:   2.5.1
Python version:    2.7.11 |Continuum Analytics, Inc.| (default, Dec  6 2015, 18:08:32) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Platform:          linux2-x86_64
Byte-ordering:     little
Detected cores:    4
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=


4

In [15] used 22.062 MiB RAM in 1.776s, peaked 0.000 MiB above current, total RAM usage 694.414 MiB


In [16]:
ca = bcolz.carray(na)

In [16] used 0.309 MiB RAM in 0.103s, peaked 0.000 MiB above current, total RAM usage 694.723 MiB


In [17]:
print("mem_used:", mw.measurements.memory_delta)

('mem_used:', 0.30859375)
In [17] used 0.008 MiB RAM in 0.004s, peaked 0.000 MiB above current, total RAM usage 694.730 MiB


Ok, this time the amount of memory used seems much lower.  Also, bcolz containers can provide an estimation on how much memory they are taking; let's have a look:

In [18]:
ca

carray((10000000,), float64)
  nbytes: 76.29 MB; cbytes: 1.01 MB; ratio: 75.31
  cparams := cparams(clevel=9, shuffle=1, cname='blosclz')
[  0.00000000e+00   1.00000000e+00   2.00000000e+00 ...,   9.99999700e+06
   9.99999800e+06   9.99999900e+06]

In [18] used 0.016 MiB RAM in 0.016s, peaked 0.000 MiB above current, total RAM usage 694.746 MiB


In this case we see that bcolz estimation is reasonably close to `ipython_memwatcher` measurements.  Let's have a look at the speed of the reduction:

In [19]:
t = %timeit -o ca.sum()
print("MFLOPS:", round(len(a) / t.best / 1e6, 3))

10 loops, best of 3: 79.8 ms per loop
('MFLOPS:', 125.344)
In [19] used 0.020 MiB RAM in 3.307s, peaked 0.000 MiB above current, total RAM usage 694.766 MiB


This is around 2~5x slower (depending on the machine) than a regular NumPy array, but the size of the array is an impressive 75x smaller.  But is compression the only responsible of the overhead?  Let's investigate a bit further.

## Using uncompressed containers with bcolz

In order to see if this is because of the compression overhead, let's use an uncompressed array:

In [20]:
cau = bcolz.carray(a, cparams=bcolz.cparams(clevel=0))

In [20] used 0.039 MiB RAM in 1.154s, peaked 0.000 MiB above current, total RAM usage 694.805 MiB


In [21]:
cau

carray((10000000,), float64)
  nbytes: 76.29 MB; cbytes: 76.52 MB; ratio: 1.00
  cparams := cparams(clevel=0, shuffle=1, cname='blosclz')
[  0.00000000e+00   1.00000000e+00   2.00000000e+00 ...,   9.99999700e+06
   9.99999800e+06   9.99999900e+06]

In [21] used 0.004 MiB RAM in 0.011s, peaked 0.000 MiB above current, total RAM usage 694.809 MiB


In [22]:
t = %timeit -o cau.sum()
print("MFLOPS:", round(len(a) / t.best / 1e6, 3))

10 loops, best of 3: 52 ms per loop
('MFLOPS:', 192.383)
In [22] used 0.008 MiB RAM in 2.206s, peaked 0.000 MiB above current, total RAM usage 694.816 MiB


As we can see, the times with an uncompressed `carray` are very close to a compressed one, so compressing is not the only source of the overhead (and it is very minor in fact).  The actual source of the difference is the memory layout of the different containers (bcolz layout is a bit more complex than NumPy).

So, bcolz allows to use compressed in-memory data containers at the cost of some performance (compared with NumPy).  But the performance overhead is rather small, and sometimes you prefer to keep more data in-memory.

On another hand, in the next notebooks we are going to see that bcolz can be pretty close to NumPy, performance wise, in some scenarios.